In [2]:
# Redmine ticket 6745 - Migrate choice options to m2m
# !IMPORTANT **Dump the data for model to json before running any migrations

# Imports
import json
import os 
import pandas as pd
from django.apps import apps as django_apps
from django.core.exceptions import ValidationError
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [7]:
file_path = 'relationshipfatherinvolvement.json'
json_file = open(file_path)
data = json.load(json_file)
json_file.close()

for item in data:
    model_obj = RelationshipFatherInvolvement.objects.get(pk=item.get('pk'))
    m2m_fields = model_obj._meta.many_to_many
    
    fields_data = item.get('fields')
    for field in m2m_fields:
        name = field.name
        # Get response from choice field
        original_response = fields_data.get(f'{name}')
        # Add related m2m response to m2m field
        related_model = field.related_model
        response_obj = related_model.objects.get(short_name=original_response)
        getattr(model_obj, f'{name}').add(response_obj)
    